In [1]:
import numpy as np
import pandas as pd
import glob
import re
from sklearn.utils import shuffle
import torch
import torch.nn.functional as F
import torch.nn as nn
from sklearn.metrics import confusion_matrix, f1_score
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [10]:
HamDirs = pd.Series(np.array(glob.glob('datasets/email/ham/*/*/*')))
SpamDirs = pd.Series(np.array(glob.glob('datasets/email/spam/*/*/*')))

SpamDirs.shape, HamDirs.shape

((21103,), (19088,))

In [11]:
SpamDirs.head()

0    datasets/email/spam/SH/HP/prodmsg.2.431622.200...
1    datasets/email/spam/SH/HP/prodmsg.2.431958.200...
2    datasets/email/spam/SH/HP/prodmsg.2.430984.200...
3    datasets/email/spam/SH/HP/prodmsg.2.444740.200...
4    datasets/email/spam/SH/HP/prodmsg.2.447580.200...
dtype: object

Email content

In [12]:
with open(SpamDirs[1], encoding='windows-1252') as f:
    file_content = f.read()

print(file_content)

Received: from c2b03-016.milare-tv.ne.jp (HELO @smtp) (219.124.83.16)
  by projecthoneypotmailserver with SMTP; 25 Jun 2005 10:59:32 -0000
Received: from 209.239.43.9 (helo=MANN)
	by smtp with esmtpa (Exim 4.50b (OpenBSD))
	id J85Gz020412586
	for projecthoneypot@projecthoneypot.org; Sat, 25 Jun 2005 10:55:18 +0000
Date: Sat, 25 Jun 2005 10:55:18 +0000
From: "Kristina" <nc46f7486j9@direcway.com>
X-Mailer: Java-x Mailer v1.3b
Reply-To: "Kristina" <nc46f7486j9@direcway.com>
X-Priority: 3 (Normal)
Message-ID: <t4hcwe.20050606213053@localhost>
To: projecthoneypot@projecthoneypot.org
Subject: Any med for your girl to be happy! 
MIME-Version: 1.0
Content-Type: text/html; charset=Windows-1252
Content-Transfer-Encoding: 8bit

<html>
<body>
Your girl is unsatisfied with your potency? Don't wait until she finds another men!<br><br>

Click <a href="http://rbccrl.skysalononline.info/?mpkuwixwtqtyudbwwczpohvwdwg">here</a> to choose from a great variety of LlCENSED love t@bs! Best pri$es ,fast shippi

#### Get Content (With Data Cleaning)

For Header Handling

In [15]:
# For Header Filter

MailHeaders = ["received", 'mime-version', 'reply-to',
             "date", "content-transfer-encoding", "message-id", "x-from", "x-to", "x-cc", "x-bcc",
             "x-origin", "x-filename", "x-priority", "x-msmail-priority", "organization", "x-mailer"]

EmailHeaderRegex = "from:\s(?P<from>.*)?[^^]*subject:\s(?P<subject>.*)?[^^]*content-type:\s(?P<content_type>.*)?"

In [16]:
# Extract Mail

def getContent(filePath):
    
    with open(filePath, encoding='windows-1252') as f:
        contents = f.read()
        
        # Data Cleaning
        contents = contents.lower()
        contents = re.sub("<[^<>]+>", " __tag__ ", contents)
        contents = re.sub("[0-9]+", "number", contents)
        contents = re.sub("(http|https)://[^\s]*", "httpaddr", contents)
        contents = re.sub("[^\s]+@[^\s]+", "emailaddr", contents)
        contents = re.sub("[$]+", "dollar", contents)
        contents = re.sub("(" + ":|".join(MailHeaders)  + ")+", "header:", contents)
        
    Email_Headers = re.findall(EmailHeaderRegex, contents)
    (Email_From, Subject, Content_Type) = Email_Headers[0] if len(Email_Headers) else ("", "", "") 
    
    __SEPERATOR__ = " __SEPERATOR__ "
    
    BodyRegex = "(?:[^^]*)(?:header\s*:.*\n)([^^]*)"  
    Body = re.findall(BodyRegex, contents)
    Body = Body[0] if len(Body) else ""
    
    
    MailMessage = re.split("-----original message-----", contents)[0]
    content = Email_From + __SEPERATOR__ + Subject + __SEPERATOR__ + Content_Type + __SEPERATOR__ + "\n" + Body
    
    contentSeries = pd.Series([filePath, content, Body, Email_From, Subject, Content_Type], 
                              index=["filePath", "content", "body", "from", "subject", "content_type"]);
    
    return contentSeries

### We will use initial 700 records for training

In [17]:
HamDf = HamDirs[:700].apply(getContent)
SpamDf = SpamDirs[:700].apply(getContent)
HamDf["label"] = 0
SpamDf["label"] = 1

SpamDf.head()

,filePath,content,body,from,subject,content_type,label
0,datasets/email/spam/SH/HP/prodmsg.2.431622.200...,"""goto my meeting"" __tag__ __SEPERATOR__ the ...",\n __tag__ \n\n __tag__ \n __tag__ \n __tag__ ...,"""goto my meeting"" __tag__",the next generation in online meetings!,text/html,1
1,datasets/email/spam/SH/HP/prodmsg.2.431958.200...,"""kristina"" __tag__ __SEPERATOR__ any med for...",\n __tag__ \n __tag__ \nyour girl is unsatisfi...,"""kristina"" __tag__",any med for your girl to be happy!,text/html; charset=windows-number,1
2,datasets/email/spam/SH/HP/prodmsg.2.430984.200...,__SEPERATOR__ __SEPERATOR__ __SEPERATOR__ \...,\n __tag__ \n __tag__ \n __tag__ numberth rebu...,,,,1
3,datasets/email/spam/SH/HP/prodmsg.2.444740.200...,"""ardith"" __tag__ __SEPERATOR__ save your mon...",\n __tag__ \n __tag__ \nneed in software for y...,"""ardith"" __tag__",save your money by getting an oem software!,text/html; charset=windows-number,1
4,datasets/email/spam/SH/HP/prodmsg.2.447580.200...,mail delivery subsystem __tag__ __SEPERATOR_...,\n\n,mail delivery subsystem __tag__,mail message detected as spam,"text/plain; charset=""utf-number""",1


In [20]:
print(SpamDf.loc[1, "body"])


 __tag__ 
 __tag__ 
your girl is unsatisfied with your potency? don't wait until she finds another men! __tag__  __tag__ 

click  __tag__ here __tag__  to choose from a great variety of llcensed love emailaddr best pridollares ,fast shipping and guaranteed effect! here you buy it riqht from warehouse! __tag__  __tag__ 

the store is verifled by bbb and approved by visa! __tag__ 

 __tag__ 
 __tag__ 



In [21]:
print(SpamDf.loc[1, "content"])

"kristina"  __tag__  __SEPERATOR__ any med for your girl to be happy!  __SEPERATOR__ text/html; charset=windows-number __SEPERATOR__ 

 __tag__ 
 __tag__ 
your girl is unsatisfied with your potency? don't wait until she finds another men! __tag__  __tag__ 

click  __tag__ here __tag__  to choose from a great variety of llcensed love emailaddr best pridollares ,fast shipping and guaranteed effect! here you buy it riqht from warehouse! __tag__  __tag__ 

the store is verifled by bbb and approved by visa! __tag__ 

 __tag__ 
 __tag__ 



##### Mail dataset with label

In [24]:
datasetDf = pd.concat([HamDf, SpamDf], axis = 0, ignore_index = True)
datasetDf = shuffle(datasetDf, random_state= 0)
datasetDf.to_csv("datasets/email/email.csv", index = False)

datasetDf.head()

,filePath,content,body,from,subject,content_type,label
1110,datasets/email/spam/SH/HP/prodmsg.2.431713.200...,"""goto my meeting"" __tag__ __SEPERATOR__ the ...",\n __tag__ \n\n __tag__ \n __tag__ \n __tag__ ...,"""goto my meeting"" __tag__",the next generation in online meetings!,text/html,1
310,datasets/email/ham/lokay-m/personal/142,emailaddr __SEPERATOR__ good job! __SEPERATOR_...,\n\nthanks to the teamwork and dedication of t...,emailaddr,good job!,text/plain; charset=us-ascii,0
776,datasets/email/spam/SH/HP/prodmsg.2.447549.200...,mail delivery system __tag__ __SEPERATOR__ j...,importance: normal\n\n __tag__ \n __tag__ \n _...,mail delivery system __tag__,just to her...,text/html; charset=iso-number-number,1
491,datasets/email/ham/lokay-m/tw_commercial_group...,"emailaddr __SEPERATOR__ tw weekly,number-numbe...",\nplease see attached file and call me if you ...,emailaddr,"tw weekly,number-number-number",text/plain; charset=us-ascii,0
1063,datasets/email/spam/SH/HP/prodmsg.2.437125.200572,"""computer shopping network"" __tag__ __SEPERA...",\n __tag__ \n __tag__ \n\n __tag__ \n\n __tag_...,"""computer shopping network"" __tag__",number's of computer products on sale now!,text/html,1


##### Vocabulary

In [25]:
# Genarate Vocab
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = datasetDf["content"].to_numpy()

def yield_tokens(data_iter):
    for content in data_iter:
        yield tokenizer(content)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

##### Tokenization of words (which words are present in content)

In [30]:
# Tokenization
def preprocessData(dfS):
    content = dfS["content"]
    
    tokensIdx = vocab(tokenizer(content))
    
    tokens = np.zeros(len(vocab), dtype="int64")
    
    tokens[tokensIdx] = 1
    
    tokens = tokens.tolist()
    data = pd.Series([dfS["filePath"], tokens], index = ["file", "tokens"])
    
    return data

In [31]:
dataset_tokens_df = datasetDf.apply(preprocessData, axis =1)
dataset_tokens_df["label"] = datasetDf["label"]
# dataset_tokens_df["tokens"] = dataset_tokens_df["tokens"].apply(lambda x: "|".join(map(str, x)))
# dataset_tokens_df.to_csv("datasets/email/emailTokens.csv", index = False)

dataset_tokens_df.head()

,file,tokens,label
1110,datasets/email/spam/SH/HP/prodmsg.2.431713.200...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
310,datasets/email/ham/lokay-m/personal/142,"[0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",0
776,datasets/email/spam/SH/HP/prodmsg.2.447549.200...,"[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...",1
491,datasets/email/ham/lokay-m/tw_commercial_group...,"[0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, ...",0
1063,datasets/email/spam/SH/HP/prodmsg.2.437125.200572,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, ...",1


In [38]:
np.array(dataset_tokens_df.loc[1, "tokens"])[0:500]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

##### Train test split

In [40]:
X = np.array(dataset_tokens_df["tokens"].to_list())
Y = dataset_tokens_df["label"].values


X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.2, random_state=0)

##### Torch model with two Linear layers

In [55]:
model = nn.Sequential(
    nn.Linear(X.shape[1], 20), 
    
    nn.ReLU(),
    nn.Linear(20, 1),
    nn.Sigmoid()
)

# Stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss_fn = nn.BCELoss()

In [56]:
# Training
def train(X, Y, epoches):
    X = torch.FloatTensor(X)
    Y = torch.FloatTensor(Y)
    N = len(Y)
    model.train()
    
    sum_loss = 0
    for epoch in range(epoches):
        
        optimizer.zero_grad()
        output = model(X).squeeze()
        loss = loss_fn(output, Y)
        
        loss.backward()
        optimizer.step()

        sum_loss += float(loss)
        
        if epoch % 100 == 0:
            print("Epoch: {:4d}\t Loss: {}".format(epoch, loss))
        
    
    print("Epoch: {:4d}\t Average Loss: {}".format(epoch, sum_loss / N))
    
    


In [57]:
# For Testing
def test(X, Y):
    X = torch.FloatTensor(X)
    Y = torch.FloatTensor(Y)
    N = len(Y)
    model.eval()
    sum_loss = 0
    
    with torch.no_grad():
        output = model(X).squeeze()
        
        loss = loss_fn(output, Y)
        
        sum_loss += float(loss)
    
    print("loss: {}".format(sum_loss / N))

In [58]:
# For predication
def predict(X):
    y_pred = (model(torch.FloatTensor(X)).squeeze().detach().numpy() > 0.5).astype("int64")
    return y_pred

In [59]:
train(X_train, y_train, 800)

Epoch:    0	 Loss: 0.6958394646644592
Epoch:  100	 Loss: 0.043261732906103134
Epoch:  200	 Loss: 0.01683998294174671
Epoch:  300	 Loss: 0.00977303646504879
Epoch:  400	 Loss: 0.006657289806753397
Epoch:  500	 Loss: 0.0049499161541461945
Epoch:  600	 Loss: 0.0038907229900360107
Epoch:  700	 Loss: 0.003177174599841237
Epoch:  799	 Average Loss: 0.02422229305770348


In [60]:
test(X_test, y_test)

loss: 5.057216621935367e-05


In [61]:
y_pred = predict(X_test)
confusion_matrix(y_test, y_pred), f1_score(y_test, y_pred)

(array([[137,   0],
        [  2, 141]]),
 0.9929577464788732)

In [62]:
y_pred = predict(X_train)
confusion_matrix(y_train, y_pred)

array([[563,   0],
       [  0, 557]])

### We will test on records after 700

In [65]:
HamDf2 = HamDirs[700:900].apply(getContent)
SpamDf2 = SpamDirs[700:800].apply(getContent)
HamDf2["label"] = 0
SpamDf2["label"] = 1

datasetDf2 = pd.concat([HamDf2, SpamDf2], axis = 0, ignore_index = True)
datasetDf2.head()

,filePath,content,body,from,subject,content_type,label
0,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ sun devil to panda lin...,"\nhi beth,\n\taccording to the engineers, mini...",emailaddr,sun devil to panda line pressure,text/plain; charset=us-ascii,0
1,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ tw bullets number/numb...,"\nfuel hedging - we hedged another number,numb...",emailaddr,tw bullets number/number,text/plain; charset=us-ascii,0
2,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ sun devil fuel __SEPER...,\nben and ron--\n\tfollowing up on our meeting...,emailaddr,sun devil fuel,text/plain; charset=us-ascii,0
3,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ red rock contract priv...,\nin response to a concern stan brought up yes...,emailaddr,red rock contract privileged and confidential,text/plain; charset=us-ascii,0
4,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ tw bullets number/numb...,\nagave energy - agave purchased a seasonal wi...,emailaddr,tw bullets number/number,text/plain; charset=us-ascii,0


In [67]:
# Shuffle our dataset
datasetDf2 = shuffle(datasetDf2, random_state= 0)
datasetDf2.head()

,filePath,content,body,from,subject,content_type,label
207,datasets/email/spam/SH/HP/prodmsg.2.444035.200...,"""quick business solutions"" __tag__ __SEPERAT...",\n __tag__ __tag__ \n __tag__ \n __tag__ \n _...,"""quick business solutions"" __tag__",investor alert spni up number%,text/html,1
217,datasets/email/spam/SH/HP/prodmsg.2.428963.200...,__SEPERATOR__ __SEPERATOR__ __SEPERATOR__ \...,importance: normal\n\n __tag__ \n __tag__ \n _...,,,,1
184,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ williams oba __SEPERAT...,"\nmichelle,\n\n\there's a draft of the letter....",emailaddr,williams oba,text/plain; charset=us-ascii,0
6,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ gomez to puckett tie-i...,\n---------------------- forwarded by kevin hy...,emailaddr,gomez to puckett tie-in strategy issues,text/plain; charset=us-ascii,0
57,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ bullets number/number ...,\nnew mexico train de-railment - most of this ...,emailaddr,bullets number/number,text/plain; charset=us-ascii,0


In [68]:
# Tokenization
dataset_tokens_df2 = datasetDf2.apply(preprocessData, axis =1)
dataset_tokens_df2["label"] = datasetDf2["label"]

In [69]:
X2 = np.array(dataset_tokens_df2["tokens"].to_list())
Y2 = dataset_tokens_df2["label"].values

X_train2, X_test2, y_train2, y_test2 = train_test_split( X2, Y2, test_size=0.2, random_state=0)

In [70]:
y_pred = predict(X_test2)
print(f1_score(y_test2, y_pred))

confusion_matrix(y_test2, y_pred)

1.0


array([[37,  0],
       [ 0, 23]])

In [71]:
y_pred = predict(X_train2)
print(f1_score(y_train2, y_pred))
confusion_matrix(y_train2, y_pred)

1.0


array([[163,   0],
       [  0,  77]])